In [1]:
#Import required functions and classes
from sahi import AutoDetectionModel
from sahi.utils.cv import read_image, read_image_as_pil
from sahi.utils.file import Path, increment_path, list_files, save_json, save_pickle, download_from_url
from sahi.predict import get_prediction, get_sliced_prediction, predict, agg_prediction, get_prediction_batched, get_sliced_prediction_batched, predict 
from sahi.prediction import visualize_object_predictions
from sahi.postprocess.utils import ObjectPredictionList, has_match, merge_object_prediction_pair
from IPython.display import Image
from numpy import asarray
import cv2
import os
import time
from PIL import Image
import json
import numpy as np
from tqdm import tqdm
import time
from sahi.prediction import ObjectPrediction, PredictionResult
from pathlib import Path
from multiprocessing import Pool, cpu_count
import torch
from torchvision.ops import nms, clip_boxes_to_image

In [2]:
yolov8_model_path = 'models/yolov8/last.pt'

In [4]:
detection_model = AutoDetectionModel.from_pretrained(
    model_type='yolov8',
    model_path=yolov8_model_path,
    confidence_threshold=0.3,
    device="cuda:0", # or 'cpu'
)

In [5]:
def get_slice_parameters(object_density, slice_size):
    
    if object_density >= 50:
        #slice_size = min_dim // 4
        slice_width = slice_size
        slice_height = slice_size
        overlap_width_ratio = 0.5
        overlap_height_ratio = 0.5
    elif 25 <= object_density < 50:
        #slice_size = min_dim // 2
        slice_width = slice_size
        slice_height = slice_size
        overlap_width_ratio = 0.25
        overlap_height_ratio = 0.25
    elif 10 <= object_density < 25:
        #slice_size = min_dim // 2
        slice_width = slice_size
        slice_height = slice_size
        overlap_width_ratio = 0.15
        overlap_height_ratio = 0.15
    else:
        return

    return slice_width, slice_height, overlap_width_ratio, overlap_height_ratio

In [6]:
def nms_merge(predictions, iou_threshold=0.5):
    if not predictions:
        return []
    boxes = torch.tensor([p.bbox.to_xyxy() for p in predictions])
    scores = torch.tensor([p.score.value for p in predictions])
    keep_indices = nms(boxes, scores, iou_threshold).tolist()
    return [predictions[i] for i in keep_indices]

In [7]:
# Function to get image details by image_id
def get_image_id(coco_data, image_name):
    for image in coco_data["images"]:
        file_name = Path(image['file_name']).stem
        if file_name == image_name:
            return image['id']
    return None

In [9]:
def merged_preds_to_coco(preds, image_id):
    coco_predictions = []
    for pred in preds:
        try:
            coco = pred.to_coco_prediction(image_id=image_id)
            coco_dict = {
                "image_id": coco.image_id,
                "bbox": coco.bbox,
                "score": coco.score,
                "category_id": coco.category_id,
                "segmentation": coco.segmentation,
                "iscrowd": coco.iscrowd,
                "area": coco.area,
            }
            # Optionally include category_name (not used in COCO eval)
            # coco_dict["category_name"] = coco.category_name

            # Only add if bbox is valid (non-empty)
            if coco_dict["bbox"]:
                coco_predictions.append(coco_dict)
        except Exception as e:
            print(f"Failed to convert to COCO dict: {e}")
    return coco_predictions


In [10]:
def predict_fine_sliced_images_refactored_nms(input_folder, dataset_json_path, detection_model, base_slice_size=512):
    name = "exp"
    save_dir = Path(increment_path(Path("sliced_predictions") / name, exist_ok=False))
    os.makedirs(save_dir, exist_ok=True)

    if dataset_json_path:
        with open(dataset_json_path, "r") as file:
            data = json.load(file)
        
    vis_params = {
        "bbox_thickness": 2,
        "text_size": 0.5,
        "text_thickness": 1,
        "hide_labels": False,
        "hide_conf": False,
        "format": "png"
    }

    image_files = [
        f for f in os.listdir(input_folder)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ]

    print(f"\nRunning fine slicing prediction on {len(image_files)} image(s)...")
    total_prediction_time = 0.0
    grand_total_predictions = 0
    all_coco_preds = []

    for filename in image_files:
        image_path = os.path.join(input_folder, filename)
        image_pil = Image.open(image_path).convert("RGB")
        image_np = np.array(image_pil)
        image_h, image_w = image_np.shape[:2]
        filename_wo_ext = Path(filename).stem
        total_prediction_count = 0
        print("*****************************************")
        print("File Name", filename_wo_ext)

        img_id = get_image_id(data, filename_wo_ext) if dataset_json_path else None
        all_object_predictions = []

        # Split image into 2x2 grid
        grid_h, grid_w = image_h // 2, image_w // 2

        for row in range(2):
            for col in range(2):
                x1, y1 = col * grid_w, row * grid_h
                x2, y2 = min(x1 + grid_w, image_w), min(y1 + grid_h, image_h)
                sub_img = image_pil.crop((x1, y1, x2, y2))
                print("Cropped Image:", x1, y1, x2, y2)

                time_start = time.time()
                base_pred = get_prediction(sub_img, detection_model)
                time_end = time.time() - time_start
                print("Initial Prediction time is: {:.2f} ms".format(time_end * 1000))

                coarse_preds = base_pred.object_prediction_list
                object_density = len(coarse_preds)
                print("Object Density:", object_density)

                slice_params = get_slice_parameters(object_density, base_slice_size)

                iteration_time = time_end

                preds = coarse_preds.copy()

                if slice_params:
                    slice_width, slice_height, overlap_w, overlap_h = slice_params
                    print("********* Slice Parameters ***********")
                    print("Slice Width: ", slice_width)
                    print("Slice Height: ", slice_height)
                    print("Overlap Width Ratio: ", overlap_w)
                    print("Overlap Height Ratio: ", overlap_h)

                    time_start_slice = time.time()
                    sliced_pred = get_sliced_prediction(
                        sub_img,
                        detection_model,
                        slice_height=slice_height,
                        slice_width=slice_width,
                        overlap_height_ratio=overlap_h,
                        overlap_width_ratio=overlap_w,
                        postprocess_type="OptNMS",
                        postprocess_match_metric="IOU",
                        postprocess_match_threshold=0.3,
                        postprocess_min_area=16,
                        verbose=0
                    )
                    time_end_slice = time.time() - time_start_slice
                    print("Sliced Prediction time is: {:.2f} ms".format(time_end_slice * 1000))
                    
                    iteration_time += time_end_slice
                    fine_preds = sliced_pred.object_prediction_list
                    print("Pre Slicing: ", len(preds))
                    preds.extend(fine_preds)
                    print("Post Slicing: ", len(preds))
                else:
                    print("Prediction time is: {:.2f} ms".format(time_end * 1000))
                    #preds = base_pred.object_prediction_list

                # Offset bounding boxes back to original image space
                #print("Total Prediction for single image: ", len(preds))
                for pred in preds:
                    pred.bbox.minx += x1
                    pred.bbox.maxx += x1
                    pred.bbox.miny += y1
                    pred.bbox.maxy += y1
                    all_object_predictions.append(pred)

                total_prediction_count += len(preds)
                total_prediction_time += iteration_time

        print("________________________________________________")

        # Merge all predictions for image
        merged_preds = nms_merge(all_object_predictions, iou_threshold=0.5)
        
        #object_prediction_list = ObjectPredictionList(all_object_predictions)
        #merged_keep = truncated_nms_merge(object_prediction_list.totensor())
        #selected_object_predictions = object_prediction_list[merged_keep].tolist()
        #if not isinstance(selected_object_predictions, list):
        #    selected_object_predictions = [selected_object_predictions]
        
        # Convert merged predictions to COCO format
        if dataset_json_path:
            coco_preds = merged_preds_to_coco(merged_preds, img_id)
            all_coco_preds.extend(coco_preds)
            
        # Visualization
        visualize_object_predictions(
            image=np.ascontiguousarray(image_pil),
            object_prediction_list=merged_preds,
            rect_th=vis_params["bbox_thickness"],
            text_size=vis_params["text_size"],
            text_th=vis_params["text_thickness"],
            hide_labels=vis_params["hide_labels"],
            hide_conf=vis_params["hide_conf"],
            output_dir=save_dir,
            file_name=filename_wo_ext,
            export_format=vis_params["format"]
        )

        grand_total_predictions += total_prediction_count

    if dataset_json_path:
        save_path = str(save_dir / "result.json")
        save_json(all_coco_preds, save_path)
   
    print(f"\nCompleted {len(image_files)} images.")
    print("Total Prediction Count (Before NMS Merge): ", grand_total_predictions)
    print("Total Prediction Count (After Final NMS Merge): ", len(all_coco_preds))
    print("Total Prediction time for all images is: {:.2f} ms".format(total_prediction_time * 1000))
    print(f"Prediction results are successfully exported to {save_dir}")


In [ ]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
# Using Different IoU threshold
source_folder = './single_test/images_bbk'
json_path = "./subset_vis_test_data_1563.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_nms(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)

In [14]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
# Using Different IoU threshold
source_folder = './single_test/images_bbk'
json_path = "./subset_vis_test_data_1563.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_nms(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 1 image(s)...
*****************************************
File Name 0000078_03171_d_0000009
Cropped Image: 0 0 680 382
Initial Prediction time is: 10.18 ms
Object Density: 25
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.25
Overlap Height Ratio:  0.25
Sliced Boxes Count: 8
POST PROCESS:  NMS
Original Prediction Count 90
Final Bounding Box Count (NMS):  9
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  23
Final Bounding Box Count (NMS):  1
Sliced Prediction time is: 82.03 ms
Pre Slicing:  25
Post Slicing:  66
Cropped Image: 680 0 1360 382
Initial Prediction time is: 7.61 ms
Object Density: 32
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.25
Overlap Height Ratio:  0.25
Sliced Boxes Count: 8
POST PROCESS:  NMS
Original Prediction Count 92
Final Bounding Box Count (NMS):  4
Final Bounding Box Cou

In [15]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1563.json' --result_json_path './sliced_predictions/exp443/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.03s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.099
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.221
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.073
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.245
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.405
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.667
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.094
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.177
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [16]:
result_predict_nms = predict(source='./single_test/images_bbk/',
                         dataset_json_path = './subset_vis_test_data_1563.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 256,
                         slice_width = 256,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/1 [00:00<?, ?it/s]

Image Name: 0000078_03171_d_0000009
Image Size:  (1360, 765)
Sliced Boxes Count: 50
POST PROCESS:  NMS
Performing prediction on 50 slices.


Performing inference on images: 100%|██████████| 1/1 [00:00<00:00,  1.61it/s]

Original Prediction Count 311
Final Bounding Box Count (NMS):  19
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  22
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  43
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  2
Prediction time is: 501.26 ms
Prediction results are successfully exported to runs/predict/exp281
Model loaded in 0.05901980400085449 seconds.
Slicing performed in 0.0015478134155273438 seconds.
Prediction performed in 0.5012593269348145 seconds.
Exporting performed in 0.04935932159423828 seconds.


In [17]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1563.json' --result_json_path './runs/predict/exp281/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.03s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.141
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.257
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.131
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.220
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.385
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.835
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.084
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.148
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [10]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
# Using Different IoU threshold
source_folder = './single_test/images'
json_path = "./subset_visdrone_test_990.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_nms(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 1 image(s)...
*****************************************
File Name 0000006_06773_d_0000018
Cropped Image: 0 0 680 382
Initial Prediction time is: 1634.91 ms
Object Density: 15
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 6
POST PROCESS:  NMS
Original Prediction Count 39
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  15
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  1
Sliced Prediction time is: 62.64 ms
Pre Slicing:  15
Post Slicing:  36
Cropped Image: 680 0 1360 382
Initial Prediction time is: 7.39 ms
Object Density: 6
Prediction time is: 7.39 ms
Cropped Image: 0 382 680 764
Initial Prediction time is: 7.50 ms
Object Density: 8
Prediction time is: 7.50 ms
Cropped Image: 680 382 1360 764
Initial Prediction time is: 7.28 ms
Object Density: 7
Prediction time is: 7.28 ms
____

In [11]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_990.json' --result_json_path './sliced_predictions/exp442/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.422
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.754
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.493
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.976
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.750
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.574
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.435
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.479
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [12]:
result_predict_nms = predict(source='./single_test/',
                         dataset_json_path = './subset_visdrone_test_990.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 256,
                         slice_width = 256,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/1 [00:00<?, ?it/s]

Image Name: 0000006_06773_d_0000018
Image Size:  (1360, 765)
Sliced Boxes Count: 50
POST PROCESS:  NMS
Performing prediction on 50 slices.


Performing inference on images: 100%|██████████| 1/1 [00:00<00:00,  1.64it/s]

Original Prediction Count 143
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  29
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  12
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  1
Prediction time is: 495.65 ms
Prediction results are successfully exported to runs/predict/exp280
Model loaded in 0.04387545585632324 seconds.
Slicing performed in 0.0015096664428710938 seconds.
Prediction performed in 0.495654821395874 seconds.
Exporting performed in 0.05056452751159668 seconds.


In [13]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_990.json' --result_json_path './runs/predict/exp280/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.391
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.704
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.426
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.958
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.741
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.287
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.402
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.468
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

#### **Subset of 50 images (V1)**

In [9]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
# Using Different IoU threshold
source_folder = './test_subsets/v6/images'
json_path = "./subset_visdrone_test_data_50_v1.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_nms(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000283
Cropped Image: 0 0 700 394
Initial Prediction time is: 1632.34 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  NMS
Original Prediction Count 68
Final Bounding Box Count (NMS):  16
Final Bounding Box Count (NMS):  15
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  4
Sliced Prediction time is: 82.92 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.58 ms
Object Density: 3
Prediction time is: 7.58 ms
Cropped Image: 0 394 700 788
Initial Prediction time is: 7.77 ms
Object Density: 16
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  NMS
Original Prediction Count 58
Final Boundi

In [10]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './sliced_predictions/exp427/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.44s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.139
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.246
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.136
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.187
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.314
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.474
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.099
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.188
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [20]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
# Using Different IoU threshold
source_folder = './test_subsets/v6/images'
json_path = "./subset_visdrone_test_data_50_v1.json"
slice_size = 128

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_nms(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000283
Cropped Image: 0 0 700 394
Initial Prediction time is: 8.89 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 28
POST PROCESS:  NMS
Original Prediction Count 112
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  45
Final Bounding Box Count (NMS):  20
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  2
Sliced Prediction time is: 197.66 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.41 ms
Object Density: 3
Prediction time is: 7.41 ms
Cropped Image: 0 394 700 788
Initial Prediction time is: 7.46 ms
Object Density: 16
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes

In [21]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './sliced_predictions/exp432/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.50s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.123
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.210
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.126
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.138
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.318
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.474
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.072
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.196
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [22]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
# Using Different IoU threshold
source_folder = './test_subsets/v6/images'
json_path = "./subset_visdrone_test_data_50_v1.json"
slice_size = 64

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_nms(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000283
Cropped Image: 0 0 700 394
Initial Prediction time is: 10.26 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 91
POST PROCESS:  NMS
Original Prediction Count 70
Final Bounding Box Count (NMS):  51
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  2
Sliced Prediction time is: 535.48 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.39 ms
Object Density: 3
Prediction time is: 7.39 ms
Cropped Image: 0 394 700 788
Initial Prediction time is: 7.51 ms
Object Density: 16
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 91
POST PROCESS:  NMS
Original Prediction Count 40
Final Bounding B

In [23]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './sliced_predictions/exp433/result.json'

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.52s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.141
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.241
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.146
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.163
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.329
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.474
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.086
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.205
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [12]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
# Using Different IoU threshold
source_folder = './test_subsets/v6/images'
json_path = "./subset_visdrone_test_data_50_v1.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_nms(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000283
Cropped Image: 0 0 700 394
Initial Prediction time is: 10.65 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  31
Final Bounding Box Count (OptNMS): 17
Adaptive Filtered Prediction:  23
Final Bounding Box Count (OptNMS): 17
Adaptive Filtered Prediction:  8
Final Bounding Box Count (OptNMS): 5
Adaptive Filtered Prediction:  6
Final Bounding Box Count (OptNMS): 4
Sliced Prediction time is: 80.96 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.45 ms
Object Density: 3
Prediction time is: 7.45 ms
Cropped Image: 0 394 700 788
Initial Prediction time is: 7.46 ms
Object Density: 16
********* Slice Parameters ***********
Slice Width:

In [13]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './sliced_predictions/exp428/result.json'

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.45s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.139
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.246
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.136
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.186
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.314
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.474
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.098
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.188
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [14]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
# Using Different IoU threshold
source_folder = './test_subsets/v6/images'
json_path = "./subset_visdrone_test_data_50_v1.json"
slice_size = 128

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_nms(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000283
Cropped Image: 0 0 700 394
Initial Prediction time is: 10.30 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 28
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  66
Final Bounding Box Count (OptNMS): 48
Adaptive Filtered Prediction:  24
Final Bounding Box Count (OptNMS): 21
Adaptive Filtered Prediction:  8
Final Bounding Box Count (OptNMS): 5
Adaptive Filtered Prediction:  9
Final Bounding Box Count (OptNMS): 8
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 2
Sliced Prediction time is: 198.79 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.41 ms
Object Density: 3
Prediction time is: 7

In [15]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './sliced_predictions/exp429/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.51s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.122
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.210
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.126
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.138
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.317
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.474
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.071
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.196
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [16]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
# Using Different IoU threshold
source_folder = './test_subsets/v6/images'
json_path = "./subset_visdrone_test_data_50_v1.json"
slice_size = 64

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_nms(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000283
Cropped Image: 0 0 700 394
Initial Prediction time is: 10.34 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 91
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  56
Final Bounding Box Count (OptNMS): 55
Adaptive Filtered Prediction:  8
Final Bounding Box Count (OptNMS): 8
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 4
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Sliced Prediction time is: 536.92 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.41 ms
Object Density: 3
Prediction time is: 7.41 ms
Cropped Image: 0 394 700 788
Initial Prediction time is: 7.49 ms
Object Density: 16
********* Slice Parameters ***********
Slice Width:  

In [17]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './sliced_predictions/exp430/result.json'

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.52s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.141
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.241
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.146
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.164
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.329
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.474
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.086
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.205
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

#### **Subset of 15 images (V5)**

In [48]:
result_predict_nms = predict(source='./test_subsets/v5',
                         dataset_json_path = './subset_visdrone_test_data_15_v5.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 512,
                         slice_width = 512,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/15 [00:00<?, ?it/s]

Image Name: 9999963_00000_d_0000057
Image Size:  (1400, 1050)
Sliced Boxes Count: 20
POST PROCESS:  NMS
Performing prediction on 20 slices.


Performing inference on images:   7%|▋         | 1/15 [00:00<00:07,  1.90it/s]

Original Prediction Count 129
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  27
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  3
Prediction time is: 387.01 ms
Image Name: 9999963_00000_d_0000006
Image Size:  (1400, 1050)
Sliced Boxes Count: 20
POST PROCESS:  NMS
Performing prediction on 20 slices.


...

Original Prediction Count 174
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  28
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Prediction time is: 248.81 ms
Image Name: 9999952_00000_d_0000216
Image Size:  (1400, 788)
Sliced Boxes Count: 15
POST PROCESS:  NMS
Performing prediction on 15 slices.


Performing inference on images:  93%|█████████▎| 14/15 [00:05<00:00,  2.69it/s]

Original Prediction Count 119
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  16
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  1
Prediction time is: 246.69 ms
Image Name: 9999938_00000_d_0000124
Image Size:  (1400, 788)
Sliced Boxes Count: 15
POST PROCESS:  NMS
Performing prediction on 15 slices.


Performing inference on images: 100%|██████████| 15/15 [00:05<00:00,  2.54it/s]

Original Prediction Count 66
Final Bounding Box Count (NMS):  10
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  4
Prediction time is: 243.37 ms
Prediction results are successfully exported to runs/predict/exp279
Model loaded in 0.05108475685119629 seconds.
Slicing performed in 0.020078182220458984 seconds.
Prediction performed in 4.10604190826416 seconds.
Exporting performed in 0.050679683685302734 seconds.


In [49]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './runs/predict/exp279/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.19s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.188
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.306
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.198
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.189
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.347
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.637
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.102
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.221
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [51]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 512

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_nms(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999963_00000_d_0000057
Cropped Image: 0 0 700 525
Initial Prediction time is: 11.65 ms
Object Density: 3
Prediction time is: 11.65 ms
Total Prediction for single image:  3
Cropped Image: 700 0 1400 525
Initial Prediction time is: 8.97 ms
Object Density: 20
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 4
POST PROCESS:  NMS
Original Prediction Count 71
Final Bounding Box Count (NMS):  22
Final Bounding Box Count (NMS):  1
Sliced Prediction time is: 74.35 ms
Pre Slicing:  20
Post Slicing:  43
Total Prediction for single image:  43
Cropped Image: 0 525 700 1050
Initial Prediction time is: 8.37 ms
Object Density: 7
Prediction time is: 8.37 ms
Total Prediction for single image:  7
Cropped Image: 700 525 1400 1050
Initial Prediction time is: 8.27 ms
Object Density: 2
Prediction time is: 

In [46]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 512

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_nms(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999963_00000_d_0000057
Cropped Image: 0 0 700 525
Initial Prediction time is: 12.53 ms
Object Density: 3
Prediction time is: 12.53 ms
Cropped Image: 700 0 1400 525
Initial Prediction time is: 9.16 ms
Object Density: 20
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 4
POST PROCESS:  NMS
Original Prediction Count 71
Final Bounding Box Count (NMS):  22
Final Bounding Box Count (NMS):  1
Sliced Prediction time is: 75.24 ms
Pre Slicing:  20
Post Slicing:  43
Cropped Image: 0 525 700 1050
Initial Prediction time is: 8.37 ms
Object Density: 7
Prediction time is: 8.37 ms
Cropped Image: 700 525 1400 1050
Initial Prediction time is: 8.32 ms
Object Density: 2
Prediction time is: 8.32 ms
________________________________________________
*****************************************
File Name 999996

In [47]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp440/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.16s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.137
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.237
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.144
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.174
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.328
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.264
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.092
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.202
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [39]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_nms(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999963_00000_d_0000057
Cropped Image: 0 0 700 525
Initial Prediction time is: 67.89 ms
Object Density: 3
Prediction time is: 67.89 ms
Cropped Image: 700 0 1400 525
Initial Prediction time is: 8.45 ms
Object Density: 20
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 12
POST PROCESS:  NMS
Original Prediction Count 31
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  18
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Sliced Prediction time is: 106.48 ms
Cropped Image: 0 525 700 1050
Initial Prediction time is: 8.49 ms
Object Density: 7
Prediction time is: 8.49 ms
Cropped Image: 700 525 1400 1050
Initial Prediction time is: 8.11 ms
Object Density: 2
Prediction time is: 8.11 ms
___________________________________

In [40]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp437/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.16s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.103
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.177
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.107
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.152
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.239
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.268
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.079
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.144
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [41]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 128

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_nms(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999963_00000_d_0000057
Cropped Image: 0 0 700 525
Initial Prediction time is: 11.62 ms
Object Density: 3
Prediction time is: 11.62 ms
Cropped Image: 700 0 1400 525
Initial Prediction time is: 8.86 ms
Object Density: 20
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 35
POST PROCESS:  NMS
Original Prediction Count 38
Final Bounding Box Count (NMS):  25
Final Bounding Box Count (NMS):  6
Sliced Prediction time is: 232.31 ms
Cropped Image: 0 525 700 1050
Initial Prediction time is: 8.34 ms
Object Density: 7
Prediction time is: 8.34 ms
Cropped Image: 700 525 1400 1050
Initial Prediction time is: 8.27 ms
Object Density: 2
Prediction time is: 8.27 ms
________________________________________________
*****************************************
File Name 9999963_00000_d_0000006
Cropped Image: 

In [42]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp438/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.17s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.130
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.208
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.139
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.127
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.311
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.268
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.065
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.195
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [44]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 64

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_nms(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999963_00000_d_0000057
Cropped Image: 0 0 700 525
Initial Prediction time is: 12.12 ms
Object Density: 3
Prediction time is: 12.12 ms
Cropped Image: 700 0 1400 525
Initial Prediction time is: 9.21 ms
Object Density: 20
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 130
POST PROCESS:  NMS
Original Prediction Count 25
Final Bounding Box Count (NMS):  22
Final Bounding Box Count (NMS):  1
Sliced Prediction time is: 754.55 ms
Pre Slicing:  20
Post Slicing:  43
Cropped Image: 0 525 700 1050
Initial Prediction time is: 8.41 ms
Object Density: 7
Prediction time is: 8.41 ms
Cropped Image: 700 525 1400 1050
Initial Prediction time is: 8.28 ms
Object Density: 2
Prediction time is: 8.28 ms
________________________________________________
*****************************************
File Name 99999

In [45]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp439/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.17s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.139
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.223
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.148
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.143
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.319
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.268
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.075
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.201
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [12]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 512

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_nms(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999963_00000_d_0000057
Cropped Image: 0 0 700 525
Initial Prediction time is: 9.89 ms
Object Density: 3
Prediction time is: 9.89 ms
Cropped Image: 700 0 1400 525
Initial Prediction time is: 8.83 ms
Object Density: 20
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 4
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  70
Final Bounding Box Count (OptNMS): 27
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Sliced Prediction time is: 73.62 ms
Pre Slicing:  20
Post Slicing:  48
Cropped Image: 0 525 700 1050
Initial Prediction time is: 8.32 ms
Object Density: 7
Prediction time is: 8.32 ms
Cropped Image: 700 525 1400 1050
Initial Prediction time is: 8.21 ms
Object Density: 2
Prediction time is: 8.21 ms
________________

In [13]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp461/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.17s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.138
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.239
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.145
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.176
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.328
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.264
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.093
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.202
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [14]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_nms(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999963_00000_d_0000057
Cropped Image: 0 0 700 525
Initial Prediction time is: 11.86 ms
Object Density: 3
Prediction time is: 11.86 ms
Cropped Image: 700 0 1400 525
Initial Prediction time is: 9.17 ms
Object Density: 20
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 12
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  26
Final Bounding Box Count (OptNMS): 21
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 1
Sliced Prediction time is: 109.03 ms
Pre Slicing:  20
Post Slicing:  45
Cropped Image: 0 525 700 1050

In [15]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp462/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.16s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.104
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.177
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.108
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.152
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.240
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.268
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.079
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.145
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [16]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 128

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_nms(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999963_00000_d_0000057
Cropped Image: 0 0 700 525
Initial Prediction time is: 11.80 ms
Object Density: 3
Prediction time is: 11.80 ms
Cropped Image: 700 0 1400 525
Initial Prediction time is: 9.25 ms
Object Density: 20
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 35
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  29
Final Bounding Box Count (OptNMS): 27
Adaptive Filtered Prediction:  9
Final Bounding Box Count (OptNMS): 9
Sliced Prediction time is: 238.40 ms
Pre Slicing:  20
Post Slicing:  56
Cropped Image: 0 525 700 1050
Initial Prediction time is: 8.43 ms
Object Density: 7
Prediction time is: 8.43 ms
Cropped Image: 700 525 1400 1050
Initial Prediction time is: 8.37 ms
Object Density: 2
Prediction time is: 8.37 ms
____________

In [17]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp463/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.17s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.130
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.208
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.138
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.126
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.311
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.268
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.064
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.195
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [18]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 64

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_nms(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999963_00000_d_0000057
Cropped Image: 0 0 700 525
Initial Prediction time is: 11.84 ms
Object Density: 3
Prediction time is: 11.84 ms
Cropped Image: 700 0 1400 525
Initial Prediction time is: 9.23 ms
Object Density: 20
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 130
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  24
Final Bounding Box Count (OptNMS): 23
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Sliced Prediction time is: 759.45 ms
Pre Slicing:  20
Post Slicing:  44
Cropped Image: 0 525 700 1050
Initial Prediction time is: 8.38 ms
Object Density: 7
Prediction time is: 8.38 ms
Cropped Image: 700 525 1400 1050
Initial Prediction time is: 8.31 ms
Object Density: 2
Prediction time is: 8.31 ms
_____________

In [19]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp464/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.17s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.139
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.223
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.147
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.142
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.319
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.268
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.075
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.201
 Average Precision  (AP) @[ IoU=0.50:0.95 | a